# Project name

#### Load Python tools and Jupyter config

In [73]:
import os
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import geopandas as gpd
import spotipy
from googleapiclient.discovery import build
from spotipy.oauth2 import SpotifyClientCredentials

In [6]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [7]:
today = pd.Timestamp("today").strftime("%Y%m%d")

---

## Fetch

#### Credentials

In [34]:
client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")

In [35]:
credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret
)
sp = spotipy.Spotify(client_credentials_manager=credentials_manager)

In [59]:
track_releases = {
    "track_1": {
        "artist": "Metro Boomin",
        "title": "Like That (Kendrick Feature)",
        "release_date": "3/22",
        "track_id": "",
    },
    "track_2": {
        "artist": "Drake",
        "title": "Push Ups",
        "release_date": "4/19",
        "track_id": "3eh51r6rFWAlGQRlHx9QnQ",
    },
    "track_3": {
        "artist": "Drake",
        "title": "Taylor Made Freestyle",
        "release_date": "4/19",
        "track_id": "",
    },
    "track_4": {
        "artist": "Kendrick Lamar",
        "title": "euphoria",
        "release_date": "4/30",
        "track_id": "77DRzu7ERs0TX3roZcre7Q",
    },
    "track_5": {
        "artist": "Kendrick Lamar",
        "title": "6:16 in LA",
        "release_date": "5/3",
        "track_id": "",
    },
    "track_6": {
        "artist": "Drake",
        "title": "Family Matters",
        "release_date": "5/3",
        "track_id": "1wFFFzJ5EsKbBWZriAcubN",
    },
    "track_7": {
        "artist": "Kendrick Lamar",
        "title": "Meet the Grahams",
        "release_date": "5/3",
        "track_id": "4S8PxReB1UiDR2F5x1lyIR",
    },
    "track_8": {
        "artist": "Kendrick Lamar",
        "title": "Not Like Us",
        "release_date": "5/4",
        "track_id": "6AI3ezQ4o3HUoP6Dhudph3",
    },
    "track_9": {
        "artist": "Drake",
        "title": "The Heart Part 6",
        "release_date": "5/5",
        "track_id": "",
    },
}

In [60]:
df_tracks = pd.DataFrame.from_dict(track_releases, orient="index")

In [61]:
track_ids = df_tracks["track_id"].to_list()

In [62]:
track_data = []

for track_id in track_ids:
    try:
        track_info = sp.track(track_id)
        track_data.append(
            {
                "track_id": track_id,
                "name": track_info["name"],
                "artist": track_info["artists"][0]["name"],
                "album": track_info["album"]["name"],
                "release_date": track_info["album"]["release_date"],
                "popularity": track_info["popularity"],
            }
        )
    except spotipy.client.SpotifyException as e:
        print(f"Error for track ID {track_id}: {str(e)}")

df_tracks_info = pd.DataFrame(track_data)

Error for track ID : http status: 400, code:-1 - Unsupported URL / URI., reason: None
Error for track ID : http status: 400, code:-1 - Unsupported URL / URI., reason: None
Error for track ID : http status: 400, code:-1 - Unsupported URL / URI., reason: None
Error for track ID : http status: 400, code:-1 - Unsupported URL / URI., reason: None


In [63]:
df_tracks_info

,track_id,name,artist,album,release_date,popularity
0,3eh51r6rFWAlGQRlHx9QnQ,Push Ups,Drake,Push Ups,2024-04-19,90
1,77DRzu7ERs0TX3roZcre7Q,euphoria,Kendrick Lamar,euphoria,2024-04-30,90
2,1wFFFzJ5EsKbBWZriAcubN,Family Matters,Drake,Family Matters,2024-05-03,79
3,4S8PxReB1UiDR2F5x1lyIR,meet the grahams,Kendrick Lamar,meet the grahams,2024-05-03,79
4,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,Not Like Us,2024-05-04,85


---

## YouTube

#### Initialize YouTube API

In [86]:
api_key = os.environ.get("YOUTUBE_KEY")
youtube = build("youtube", "v3", developerKey=api_key)

#### Define each artist's YouTube channel ID

In [87]:
# https://www.youtube.com/@kendricklamar
kendrick_videos = get_channel_videos("UC3lBXcrKFnFAFkfVk5WuKcQ")
# https://www.youtube.com/@DrakeOfficial
drake_videos = get_channel_videos("UCByOQJjav0CUDwxCk-jVNRQ")

#### Function to get videos from a channel

In [93]:
def get_channel_videos(channel_id):
    # Get the upload playlist ID for the channel
    res = youtube.channels().list(id=channel_id, part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

    videos = []
    next_page_token = None

    # Fetch all videos in the playlist
    while True:
        res = (
            youtube.playlistItems()
            .list(
                playlistId=playlist_id,
                part="snippet",
                maxResults=50,  # Adjusted as needed
                pageToken=next_page_token,
            )
            .execute()
        )
        videos += res["items"]
        next_page_token = res.get("nextPageToken")

        if next_page_token is None:
            break

    return videos

#### Create dataframe with video lists

In [ ]:
video_data = []
for video in kendrick_videos + drake_videos:
    video_info = {
        "title": video["snippet"]["title"],
        "published_at": video["snippet"]["publishedAt"],
        "description": video["snippet"]["description"],
        "video_id": video["snippet"]["resourceId"]["videoId"],
    }
    video_data.append(video_info)

In [90]:
df_videos = pd.DataFrame(video_data)

In [92]:
df_videos.head()

,title,published_at,description,video_id
0,Not Like Us,2024-05-04T23:50:43Z,,T6eK-2OQtew
1,meet the grahams,2024-05-04T03:58:26Z,,2QiFl9Dc7D0
2,euphoria,2024-04-30T15:23:30Z,,NPqDIwWMtxg
3,Kendrick Lamar - Count Me Out,2022-12-16T18:00:14Z,"Kendrick Lamar “Count Me Out” \nDirected by Dave Free & Kendrick Lamar\n \nProduction Company: pgLang / project3\nExecutive Producers: Kendrick Lamar, Dave Free\nProducers: Jason Baum, Jamie Rabineau\nDirector of Photography: Adam Newport-Berra\nAdditional Photography: Christopher Ripley\n1st AD: Kenneth Taylor\nProduction Designer: Eric Hersey\nWardrobe Stylist: Taylor McNeill\nAdditional Wardrobe Styling: Shannon Stokes\nChoreographer: Charm La’Donna\nStunts: Chris Gann\nGrooming: Jenn Hanching\nMakeup: Emily Cheng\nHair Stylist: Nikki Nelms\nHair Stylist: Khristien “Khristn” Ray\nBarber: Victor Arellano\nMakeup Artist / Helen Mirren: Jo Strettell\nHair Stylist / Helen Mirren: Richard Collins\nWardrobe Stylist / Helen Mirren: Lee Harris\nColorist: Mikey Rossiter\nVFX: Tryptyc\nSound Design: Brandon Loulias\nEditor: Neal Farmer\nCommissioner: Michelle An\n \nSpecial Thanks: \nDame Helen Mirren\nJack Dorsey \nVivi Nevo\n \nThank you to our team at pgLang.\n \nKendrick Lamar “Mr. Morale & The Big Steppers” is available now:\nhttps://KendrickLamar.lnk.to/MrMorale\nhttps://oklama.com\nhttps://www.instagram.com/kendricklamar\nhttps://twitter.com/kendricklamar\nhttps://www.facebook.com/kendricklamar\n \n#KendrickLamar #CountMeOut #MrMorale #TheBigSteppers",5GhhVHpPR_M
4,Kendrick Lamar - Rich Spirit,2022-11-16T18:00:02Z,"Kendrick Lamar “Rich Spirit” \nDirected by Calmatic\n\nProduction Company: pgLang, Prettybird\nExecutive Producers / pgLang: Kendrick Lamar, Dave Free\nExecutive Producer / Prettybird: Candice Dragonas \nProducers: Brendan Garrett, Jamie Rabineau \nDirector of Photography: Chris Ripley\nProduction Designer: Scott Falconer \nChoreographer: Charm La’Donna\n1st AD: Jey Wada\nStylist: Taylor McNeill\nEditor: Grason Caldwell @ Parallax\nColor: Michael Rossiter @ Rare Medium\nVFX: The Mill \nCommissioner: Michelle An\n\nThank you to our team at pgLang\n\nKendrick Lamar “Mr. Morale & The Big Steppers” is available now: https://KendrickLamar.lnk.to/MrMorale\nhttps://oklama.com\nhttps://www.instagram.com/kendricklamar\nhttps://twitter.com/kendricklamar\nhttps://www.facebook.com/kendricklamar\n\n#KendrickLamar #RichSpirit #MrMorale #TheBigSteppers",toBTPGfurLc


In [76]:
def get_video_statistics(video_id, api_key):
    url = f"https://www.googleapis.com/youtube/v3/videos?part=statistics&id={video_id}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    return (
        data["items"][0]["statistics"]
        if "items" in data and len(data["items"]) > 0
        else None
    )

In [ ]:
# Example usage

video_id = "J_tchksaJCM"
video_stats = get_video_statistics(video_id, api_key)
print(video_stats)

{'viewCount': '1663139', 'likeCount': '84199', 'favoriteCount': '0', 'commentCount': '2864'}


---

## Exports

#### JSON

In [17]:
# df.to_json(
#     f"data/processed/NAME.json",
#     indent=4,
#     orient="records",
# )

#### CSV

In [18]:
# df.to_csv(
#     f"data/processed/NAME.csv", index=False
# )